### Sequence-to-Sequence LSTM: Predicting the Second Sentence

The LSTM model implemented in this repo draws from the [Sequence to Sequence Learning with Neural Networks (2014)](https://arxiv.org/pdf/1409.3215) paper from 2014 written by Google researchers Ilya Sutskever, Oriol Vinyals, and Quoc V. Le. The original paper uses an encoder/decoder LSTM model to convert English to French. Ben Trevett, an NLP engineer, implemented a similar but simplified seq-2-seq model based on the Google paper. There are many code segments that I'm using from Ben's implementation that can be found in his GitHub repo [here](https://github.com/bentrevett/pytorch-seq2seq).

The main difference between my code and Ben's code is that I'll be implementing a version of `nn.LSTM` myself, rather than using pytorch's pre-built implementation. Some additional differences in my implementation:

- Source vocabulary and target vocabulary are identical (as nothing is being translated)
- Doing all dataset creation/preprocessing myself
- Running everything on CPU vs. GPU

I also found a handful of other articles/videos really useful:

- [PyTorch LSTM documentation]("https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html"): This was useful in understanding what my LSTMCell class needed to have in order to function with my Encoder, Decoder, and S2S classes as well as working with PyTorch's autodifferentiation, SGD, and cross entropy loss functions.

- [StatQuest YouTube Channel]("https://www.youtube.com/watch?v=L8HKweZIOmg&t=24s&ab_channel=StatQuestwithJoshStarmer"): StatQuest YouTube channel has a bunch of videos on sequence to sequence problems, which helped with the intuitive understanding. He also has videos that implement seq-2-seq using PyTorch Lightning. He follow's Ben's repository as well.

- [Mustafa Murat ARAT Blog]("https://mmuratarat.github.io/2019-01-19/dimensions-of-lstm"): This article was helpful in understanding the dimensionality of weights/biases


#### Imports:

In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import torch.optim as optim
import tqdm
import evaluate

#TODO: Change embedding size from 50 to 200
EMBED_SIZE = 50

/home/will/miniconda3/envs/pycourse/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/will/miniconda3/envs/pycourse/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/will/miniconda3/envs/pycourse/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


#### Set all random seeds:

In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

#### Pre-process the Training Data

training_firsts (and seconds) will be a lists of length 6036 each. Each element of these lists is a sentence vector, where each element corresponds to the words in the sentence. both `<s>` and `</s>` are included in each sentence vector. 

In [3]:
# TRAINING
training_size = 6036
training_firsts = []
training_secs = []
with open("bobsue-data/bobsue.seq2seq.train.tsv") as trainfile:
    for line in trainfile:
        separated = line.split()
        for i in range(len(separated)):
            if separated[i] == "</s>":
                sent1 = separated[:i + 1]
                sent2 = separated[i + 1:]
                break
        training_firsts.append(sent1)
        training_secs.append(sent2)


Now, we repeat for validation and testing sets:

In [4]:
# VALIDATION
validation_size = 750
valid_firsts = []
valid_secs = []
with open("bobsue-data/bobsue.seq2seq.dev.tsv") as valfile:
    for line in valfile:
        separated = line.split()
        for i in range(len(separated)):
            if separated[i] == "</s>":
                sent1 = separated[:i + 1]
                sent2 = separated[i + 1:]
                break
        valid_firsts.append(sent1)
        valid_secs.append(sent2)

# TESTING
testing_size = 750
test_firsts = []
test_secs = []
with open("bobsue-data/bobsue.seq2seq.test.tsv") as testfile:
    for line in testfile:
        separated = line.split()
        for i in range(len(separated)):
            if separated[i] == "</s>":
                sent1 = separated[:i + 1]
                sent2 = separated[i + 1:]
                break
        test_firsts.append(sent1)
        test_secs.append(sent2)


#### Generating Embeddings: GloVe

We now have to give each word in our target vocabulary an embedding. To do this, we'll use GloVe. I'll be using snippits from [Martín Pellarolo’s Blog](https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76) in my implementation. `word_2_index` maps each word in the GloVe file to an index (i.e. 0, 1, ...). Associated with each index is the corresponding entry in `embeddings_vector`. `embeddings_vector` has shape `(size of glove vocabulary, EMBED_SIZE)`. `EMBED_SIZE` will eventually be 200, but for now it is 50 to speed up training. `glove_voc` stores each word in the GloVe vocabulary. Lastly, we'll create a dictionary `glove` that has keys that are vocabulary words, and the values are the corresponding embedding vectors. 

In [5]:
#GloVe
#See descriptions above for the below three variables
word_to_index = {}
embeddings_vector = []
glove_voc = []

index = 0
with open(f'embedding-data/glove.6B.{EMBED_SIZE}d.txt') as embedfile:
    for line in embedfile:
        #Store the first element of the line as the word, store the remaining elements as the embedding vector of size EMBED_SIZE
        split_vector = line.split()
        cur_embedded_word = split_vector[0]
        
        cur_embedded_vector = np.array(split_vector[1:]).astype(np.float32)
        word_to_index[cur_embedded_word] = index

        glove_voc.append(cur_embedded_word)
        embeddings_vector.append(cur_embedded_vector)
        index += 1

glove = {w: embeddings_vector[word_to_index[w]] for w in glove_voc}
index_to_add = index

We'll generate our target vocabulary from our `bobsue.voc.txt` file

In [6]:
#Pre-process target vocabulary
target_voc = []
with open('bobsue-data/bobsue.voc.txt') as vocfile:
    for line in vocfile:
        line = line.split()
        target_voc.append(line[0])
target_voc = np.array(target_voc)

The next step is to ensure that we have embeddings for all words in our target vocabulary. To do this, we'll loop through each word in `target_voc`, and if the word appears in our glove dictionary with an embedding, we link that same embedding in the new `target_embeddings` dictionary. If not, we assign a normally distributed random vector of dimension EMBED_SIZE. `target_voc` is a `numpy` array where each element is a word that appears in our training/testing data.

In [7]:
tgt_vocab_len = len(target_voc)
target_embeddings = {}
for i in range(tgt_vocab_len):
    cur_word = target_voc[i]
    if cur_word in glove:
        target_embeddings[cur_word] = glove[cur_word]
    else:
        target_embeddings[cur_word] = np.random.normal(scale=0.6, size=(EMBED_SIZE, ))
        word_to_index[cur_word] = index_to_add
        index_to_add += 1

Create a dictionary `target_to_index` from a target word to its corresponding index (does the inverse of calling `target_voc[i]`). Then, we'll define `tokenized_to_indices` to convert a tokenized sentence into a list of integers corresponding to the index in our vocabulary.

In [8]:
target_to_index = {}
index_to_target = {}
index_to_embedding = []
for i, word in enumerate(target_voc):
    target_to_index[word] = i
    index_to_target[i] = word

for i in range(len(target_voc)):
    target = index_to_target[i]
    embedding = target_embeddings[target]
    index_to_embedding.append(embedding)

index_to_embedding = np.array(index_to_embedding)
index_to_embedding = torch.from_numpy(index_to_embedding).float()


In [9]:
def tokenized_to_indices(tokens):
    indices = []
    for i in range(len(tokens)):
        indices.append(target_to_index[tokens[i]])
    return indices

We need to convert all of our training sentences into tensors of numerical embeddings. First we'll write a function that takes a tokenized sentence and outputs the corresponding tensor.

In [10]:
def sent_to_embed(tokenized_seq):
    """
    Given a tokenized sentence (where tokenized seq is a vector of tokens for a given sentence),
    returns the a vector where each element corresponds to the embedding of that word
    """
    embeddings = []
    for token in tokenized_seq:
        cur_embedding = target_embeddings[token]
        embeddings.append(cur_embedding)
    embeddings = np.array(embeddings)
    embeddings = torch.from_numpy(embeddings).float()
    return embeddings


### Aggregating our data:
Now, we'll create full datasets for training, validation, and testing.

In [11]:
#Training Data
training_data = []
for i in range(len(training_firsts)):
    cur_pair_data = {
        'src_tokens' : training_firsts[i],
        'tgt_tokens' : training_secs[i],

        'src_len' : len(training_firsts[i]),
        'tgt_len' : len(training_secs[i]),

        'src_indices' : torch.tensor(tokenized_to_indices(training_firsts[i]), dtype=torch.int16),
        'tgt_indices' : torch.tensor(tokenized_to_indices(training_secs[i]), dtype=torch.int16),

        'src_embeddings' : sent_to_embed(training_firsts[i]),
        'tgt_embeddings' : sent_to_embed(training_secs[i])
        }
    training_data.append(cur_pair_data)

We'll repeat the process for validation and testing:

In [12]:
#VALIDATION DATA
validation_data = []
for i in range(len(valid_firsts)):
    cur_pair_data = {
        'src_tokens' : valid_firsts[i],
        'tgt_tokens' : valid_secs[i],

        'src_len' : len(valid_firsts[i]),
        'tgt_len' : len(valid_secs[i]),

        'src_indices' : torch.tensor(tokenized_to_indices(valid_firsts[i]), dtype=torch.int16),
        'tgt_indices' : torch.tensor(tokenized_to_indices(valid_secs[i]), dtype=torch.int16),

        'src_embeddings' : sent_to_embed(valid_firsts[i]),
        'tgt_embeddings' : sent_to_embed(valid_secs[i])
        }
    validation_data.append(cur_pair_data)

In [13]:
#TESTING DATA
testing_data = []
for i in range(len(test_firsts)):
    cur_pair_data = {
        'src_tokens' : test_firsts[i],
        'tgt_tokens' : test_secs[i],

        'src_len' : len(test_firsts[i]),
        'tgt_len' : len(test_secs[i]),

        'src_indices' : torch.tensor(tokenized_to_indices(test_firsts[i]), dtype=torch.int16),
        'tgt_indices' : torch.tensor(tokenized_to_indices(test_secs[i]), dtype=torch.int16),

        'src_embeddings' : sent_to_embed(test_firsts[i]),
        'tgt_embeddings' : sent_to_embed(test_secs[i])
        }
    testing_data.append(cur_pair_data)

### Building the Model
We've now prepared our training, validation, and testing data. We now need to build the LSTM model, Encoder, and Decoder.

In [14]:
from model import Encoder, Decoder, S2S

#See model.py for encoder, decoder, and seq-2-seq classes

encoder = Encoder(EMBED_SIZE, EMBED_SIZE)
decoder = Decoder(EMBED_SIZE, EMBED_SIZE, index_to_embedding)
s2s = S2S(encoder, decoder)

Initalize weights: the paper uses uniform between -0.08 and 0.08. Set up optimizer and loss functions: I'm using SGD and cross entropy loss.

In [15]:
def init_weights(model):
    for name, parameter in model.named_parameters():
        nn.init.uniform_(parameter.data, -0.08, 0.08)

### Training the Model

Training Loop:

In [16]:
def train_fn(model, optimizer, criterion, training_data):
    model.train()
    epoch_loss = 0
    for i in range(len(training_data)):
        src = training_data[i]['src_embeddings']
        tgt = training_data[i]['tgt_embeddings']
        
        optimizer.zero_grad()
        output = model(src, tgt, tf=True)
        
        loss = criterion(output, tgt)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss/len(training_data)

Evaluation Loop: (no updating parameters)

In [17]:
def eval_fn(model, criterion, val_data):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i in range(len(val_data)):
            src = val_data[i]['src_embeddings']
            tgt = val_data[i]['tgt_embeddings']

            output = model(src, tgt, tf=False)

            loss = criterion(output, tgt)
            epoch_loss += loss.item()
    
    return epoch_loss/len(val_data)

Model Training:

In [18]:
s2s.apply(init_weights)
optimizer = optim.Adam(s2s.parameters())
criterion = nn.MSELoss()

n_epochs = 20
best_valid_loss = float("inf")

#Testing with subsets
#subset_size = 1000
#train_subs = training_data[:subset_size]
#valid_subs = validation_data[:subset_size]

training_losses = []
valid_losses = []

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(s2s, optimizer, criterion, training_data)
    valid_loss = eval_fn(s2s, criterion, validation_data)

    training_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f"\tTrain Loss: {train_loss}")
    print(f"\tValid Loss: {valid_loss}")
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(s2s.state_dict(), "saved_lstm.pt")   

  5%|▌         | 1/20 [00:44<14:00, 44.21s/it]

	Train Loss: 0.1745582824749434
	Valid Loss: 0.1540367602010568


 10%|█         | 2/20 [01:27<13:10, 43.93s/it]

	Train Loss: 0.1517414031408337
	Valid Loss: 0.15026627372701962


 15%|█▌        | 3/20 [02:11<12:27, 43.97s/it]

...

	Train Loss: 0.14857138955633595
	Valid Loss: 0.1485283645292123


 95%|█████████▌| 19/20 [13:48<00:43, 43.28s/it]

	Train Loss: 0.14856113900533569
	Valid Loss: 0.14852927565574647


100%|██████████| 20/20 [14:32<00:00, 43.63s/it]

	Train Loss: 0.1485492648523501
	Valid Loss: 0.1485118014315764


In [19]:
def convert_to_english(embeddings, index_to_embedding, index_to_target):
    tokens = []
    for i in range(len(embeddings)):
        minimum = np.Inf
        minimum_index = -1
        for j in range(len(index_to_embedding)):
            if torch.norm(index_to_embedding[j] - embeddings[i]) < minimum:
                minimum = torch.norm(index_to_embedding[j] - embeddings[i])
                minimum_index = j
        tokens.append(index_to_target[minimum_index])
    
    return tokens

In [20]:
def test(model, test_data, index_to_embedding, index_to_target):
    model.eval()
    epoch_loss = 0
    predicted_tokens = []
    actual_tokens = []
    with torch.no_grad():
        for i in tqdm.tqdm(range(10)): #TODO::: CHANGE MEEEE
            src = test_data[i]['src_embeddings']
            tgt = test_data[i]['tgt_embeddings']

            output = model(src, tgt, tf=False)
            predicted_tokens.append(convert_to_english(output, index_to_embedding, index_to_target))
            actual_tokens.append(convert_to_english(test_data[i]['tgt_embeddings'], index_to_embedding, index_to_target))

            loss = criterion(output, tgt)
            epoch_loss += loss.item()
    
    return epoch_loss/len(test_data), predicted_tokens, actual_tokens

test_loss, predictions, actual = test(s2s, testing_data, index_to_embedding, index_to_target)

100%|██████████| 10/10 [00:02<00:00,  4.50it/s]


Plotting Train/Validation error as a function of epoch:

In [24]:
import matplotlib as plt
epochs_x = np.arange(n_epochs) + 1

fig, ax = plt.subplots()
ax.plot(epochs_x, training_losses, label="Training Loss")
ax.plot(epochs_x, valid_losses, label="Validation Loss")



AttributeError: module 'matplotlib' has no attribute 'subplot'